In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./movie_tags_in_database.csv')

In [4]:
# import openai
# client = openai.Client(api_key="sk-proj-LjelMrYHxCiy2InWAc2QT3BlbkFJRsSNDNahsnVJEN5r0J0B")

### Prompt generating using Prediction Guard

In [6]:
import json
import predictionguard as pg
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator


In [7]:
def load_token():
  with open("../../config/predictionguard.json", 'r') as file:
    config = json.load(file)
    return config['api_key']

In [8]:
class Prompt(BaseModel):
    prompt: str = Field(
        ...,
        title="Prompt",
        description="The prompt to be used for generating the completion.",
    )   

In [9]:
parser = PydanticOutputParser(pydantic_object=Prompt)

In [10]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [11]:
result = pg.Completion.create(
    model="Neural-Chat-7B",
    prompt=prompt.format(query="Generate a question to ask about movie related to this tag: " + "spoof"),
    max_tokens=200,
    temperature=0.1
)
try:
    questions = Prompt.parse_raw(result['choices'][0]['text'])
    print(f"Prompt: {questions.prompt}")
except Exception as e:
    print(f"Error parsing joke: {e}")

ValueError: No access token provided and no predictionguard config file found. Please provide the access token as arguments or set the environment variable PREDICTIONGUARD_TOKEN.